In [1]:
import os

os.chdir('../../')

In [2]:
import pandas as pd

from collections import Counter

df_chunks = pd.read_parquet('data/s2orc/chunk_meme_mappings/reduced_300_big_cleaned_mini_all-MiniLM-L6-v2_eps_0.0.parquet')

def get_meme_content(meme_id):
    return list(df_chunks[df_chunks['meme_id']==meme_id]['chunk'])

In [3]:
meme_to_cluster_size = df_chunks.groupby('meme_id').count()['chunk']

In [4]:
df = pd.read_parquet('data/s2orc/results/reduced_300_big_cleaned_mini_all-MiniLM-L6-v2_eps_0.0.parquet')

In [5]:
all_memes = df['memes'].explode()
all_inbound_memes = df['inbound_memes'].explode()
all_outbount_memes = df['outbound_memes'].explode()
memes_count = Counter(all_memes)
inbound_memes_count = Counter(all_inbound_memes)
outbound_memes_count = Counter(all_outbount_memes)

In [6]:
from tqdm import tqdm

d = {"meme_id":[], "count":[],"inbound_count":[], "outbound_count":[],'cluster_size':[]}
for meme_id in tqdm(memes_count):
    try:
        d['meme_id'].append(int(meme_id))
    except ValueError:
        continue
    d['count'].append(memes_count[meme_id])
    d['inbound_count'].append(inbound_memes_count[meme_id])
    d['outbound_count'].append(outbound_memes_count[meme_id])
    d['cluster_size'].append(meme_to_cluster_size[meme_id])
df_memes = pd.DataFrame(d)

100%|██████████| 1073135/1073135 [00:05<00:00, 206241.59it/s]


In [7]:
df_memes.sort_values(by='count', ascending=False, inplace=True)
df_memes

,meme_id,count,inbound_count,outbound_count,cluster_size
36,950121,208335,757563,636102,1
158,948694,84431,283865,247118,1
16,235449,69884,259153,256262,1
31,762593,56753,240133,209860,1
348,1192669,52922,198857,188971,1
...,...,...,...,...,...
507025,863843,1,2,5,1
507026,1150060,1,2,5,1
507027,273515,1,2,5,1
507028,142318,1,2,5,1


In [8]:
outliers = df_memes[df_memes['cluster_size']==1]['meme_id']
df_outliers = df_chunks[df_chunks['meme_id'].isin(outliers)]
df_outliers_to_remove = df_outliers[df_outliers['chunk'].str.split(' ').str.len()<2]
memes_to_remove = df_outliers_to_remove['meme_id']

In [9]:
df_memes = df_memes[~df_memes['meme_id'].isin(memes_to_remove)]
print("With 0 outbound:")
print((df_memes['outbound_count']==0).sum())

print("With 0 inbound:")
print((df_memes['inbound_count']==0).sum())

With 0 outbound:
452371
With 0 inbound:
295090


## Calculating most common pairs cited_meme -> meme

In [10]:
df2 = df.explode("memes")
df2 = df2[['memes', 'inbound_memes']]#, 'institutions', 'countries', 'types', 'is_big_tech']]
df2.dropna(subset=['memes'], inplace=True)
len(df2)

13125108

In [11]:
df2 = df2[~df2['memes'].isin(memes_to_remove)]
df2=df2[df2['inbound_memes'].str.len()!=0]
len(df2)

4168952

In [15]:
df2 = df2[:500000]

In [16]:
df3 =df2.explode("inbound_memes")

In [21]:
out = df3.value_counts()

memes    inbound_memes
561977   950121           13761
317206   950121           10152
1168193  950121            9723
919556   950121            7304
48203    950121            6754
                          ...  
65433    58131                1
         58179                1
         58507                1
         58515                1
1306144  1306144              1
Length: 77613050, dtype: int64

In [28]:
out.reset_index(name='count').to_parquet("memes_edges.parquet")

In [13]:
# flow_count = Counter()
# def process_row(row):
#     meme = row['memes']
#     for in_meme in row['inbound_memes']:
#         flow_count[(meme, in_meme)] += 1  

# a = df2.apply(process_row, axis=1)

KeyboardInterrupt: 